# Tracking Bugs

_Brief abstract/introduction/motivation.  State what the chapter is about in 1-2 paragraphs._
_Then, have an introduction video:_

In [ ]:
from bookutils import YouTubeVideo
YouTubeVideo("w4u5gCgPlmg")

**Prerequisites**

* _Refer to earlier chapters as notebooks here, as here:_ [Earlier Chapter](Fuzzer.ipynb).

In [ ]:
import bookutils

## Synopsis

<!-- Automatically generated. Do not edit. -->



_For those only interested in using the code in this chapter (without wanting to know how it works), give an example.  This will be copied to the beginning of the chapter (before the first section) as text with rendered input and output._

You can use `int_fuzzer()` as:

```python
print(int_fuzzer())
```
```python
=> 76.5

```


## Tracking Failures

\todo{Add}

## Mining Past Changes

We can mine past changes.

In [ ]:
from pydriller import RepositoryMining

In [ ]:
from ChangeDebugger import diff, patch, print_patch

In [ ]:
# n = 10
# for commit in miner.traverse_commits():
#     print(commit.hash, commit.author.name)
#     for m in commit.modifications:
#         print(m.new_path)
#         old_version = m.source_code_before if m.source_code_before else ""
#         new_version = m.source_code if m.source_code else ""
#         patches = diff(old_version, new_version)
#         for patch in patches:
#             print_patch(patch)
#         n -= 1
#         if n <= 0:
#             break
#     if n <= 0:
#         break

In [ ]:
import pickle

In [ ]:
REPO_CACHE = "repocache.pickle"

In [ ]:
def all_modifications(repo):
    try:
        with open(REPO_CACHE, 'rb') as f:
            all_modifications = pickle.load(f)
    except FileNotFoundError:
        all_modifications = {}
    except EOFError:
        all_modifications = {}
    except TypeError:
        all_modifications = {}
    except pickle.PickleError:
        all_modifications = {}

    if repo not in all_modifications:
        miner = RepositoryMining(repo)
        all_modifications[repo] = []

        for commit in miner.traverse_commits():
            for m in commit.modifications:
                m.hash = commit.hash
                m.committer = commit.committer
                m.committer_date = commit.committer_date
                m.msg = commit.msg
                all_modifications[repo].append(m)

        with open(REPO_CACHE, 'wb') as f:
            pickle.dump(all_modifications, f)

    return all_modifications[repo]

In [ ]:
REPO = 'https://github.com/uds-se/debuggingbook.git'
# REPO = '..'

In [ ]:
all_mods = all_modifications(REPO)

In [ ]:
changes = {}
change_msgs = {}

fixes = {}
fix_msgs = {}

sizes = {}

for m in all_mods:
    if not m.new_path:
        continue
    
    node = tuple(m.new_path.split('/'))
    
    def add_node(nodes, node_msgs):
        nodes.setdefault(node, set())
        node_msgs.setdefault(node, "")
        if m.hash not in nodes[node]:
            nodes[node].add(m.hash)
            if node_msgs[node]:
                node_msgs[node] += '<br>' + m.msg
            else:
                node_msgs[node] = m.msg
    
    sizes[node] = len(m.source_code) if m.source_code else 0
    
    add_node(changes, change_msgs)
    if m.msg.startswith("Fix:") and not "show output" in m.msg:
        add_node(fixes, fix_msgs)

changes = {node: len(changes[node]) for node in changes}
fixes = {node: len(fixes[node]) for node in fixes}

In [ ]:
list(changes.keys())[:10]

In [ ]:
changes[('Chapters.makefile',)]

In [ ]:
fixes[('Chapters.makefile',)]

In [ ]:
fix_msgs[('Chapters.makefile',)]

In [ ]:
change_msgs[('Chapters.makefile',)]

In [ ]:
sizes[('Chapters.makefile',)]

## Past Changes

Now, visualize treemap (notebooks/) and give it 

* a size, depending on the number of lines;
* a color, depending on the total number of changes.

https://github.com/mwouts/easyplotly

In [ ]:
import easyplotly as ep
import plotly.graph_objects as go

In [ ]:
import math

In [ ]:
log_sizes = {node: math.log(sizes[node]) if sizes[node] else 0
             for node in sizes}

In [ ]:
def node_color(node):
    if not node:
        return
    if node in fixes:
        val = fixes[node]
        return val

In [ ]:
treemap = ep.Treemap(log_sizes,
                     text=changes,
                     hoverinfo='label',
                     marker_colors=node_color,
                     marker_colorscale='YlOrRd',
                     root_label=REPO,
                     branchvalues="total"
                    )

fig = go.Figure(treemap)

fig.update_layout(
    margin=dict(l=0, r=0, t=30, b=0),
)

## Past Fixes

In [ ]:
fixed_log_sizes = {node: log_sizes[node] for node in log_sizes 
                   if node in fixes and fixes[node] > 0}

In [ ]:
treemap = ep.Treemap(fixed_log_sizes,
                     text=fix_msgs,
                     hoverinfo='label',
                     marker_colors=node_color,
                     marker_colorscale='YlOrRd',
                     root_label=REPO,
                     branchvalues="total"
                    )

fig = go.Figure(treemap)

fig.update_layout(
    margin=dict(l=0, r=0, t=30, b=0),
)

## Fine-Grained Changes

In [ ]:
fixed_log_sizes = {node: log_sizes[node] for node in log_sizes 
                   if node in fixes and fixes[node] > 0}

In [ ]:
treemap = ep.Treemap(fixed_log_sizes,
                     text=fix_msgs,
                     hoverinfo='label',
                     marker_colors=node_color,
                     marker_colorscale='YlOrRd',
                     root_label=REPO,
                     branchvalues="total"
                    )

fig = go.Figure(treemap)

fig.update_layout(
    margin=dict(l=0, r=0, t=30, b=0),
)

Now, visualize treemap (notebooks/) and give it 

* a size, depending on the number of lines;
* a color, depending on the total number of changes.

## Synopsis

_For those only interested in using the code in this chapter (without wanting to know how it works), give an example.  This will be copied to the beginning of the chapter (before the first section) as text with rendered input and output._

## Lessons Learned

* _Lesson one_
* _Lesson two_
* _Lesson three_

## Next Steps

_Link to subsequent chapters (notebooks) here, as in:_

* [use _mutations_ on existing inputs to get more valid inputs](MutationFuzzer.ipynb)
* [use _grammars_ (i.e., a specification of the input format) to get even more valid inputs](Grammars.ipynb)
* [reduce _failing inputs_ for efficient debugging](Reducer.ipynb)


## Background

_Cite relevant works in the literature and put them into context, as in:_

The idea of ensuring that each expansion in the grammar is used at least once goes back to Burkhardt \cite{Burkhardt1967}, to be later rediscovered by Paul Purdom \cite{Purdom1972}.

## Exercises

_Close the chapter with a few exercises such that people have things to do.  To make the solutions hidden (to be revealed by the user), have them start with_

```
**Solution.**
```

_Your solution can then extend up to the next title (i.e., any markdown cell starting with `#`)._

_Running `make metadata` will automatically add metadata to the cells such that the cells will be hidden by default, and can be uncovered by the user.  The button will be introduced above the solution._

### Exercise 1: _Title_

_Text of the exercise_

In [ ]:
# Some code that is part of the exercise
pass

_Some more text for the exercise_

**Solution.** _Some text for the solution_

In [ ]:
# Some code for the solution
2 + 2

_Some more text for the solution_

### Exercise 2: _Title_

_Text of the exercise_

**Solution.** _Solution for the exercise_